# Projet 5MA-OSI : Transfert d'organes en domino sous incertitudes

## Introduction

Malgré l'augmentation croissante du nombre de transplantations d'organes effectuées chaque année (environ 6000 en 2017 dont 3782 transplantations de reins), la demande reste en perpétuelle augmentation. Ainsi 6000 organes, dont 3782 reins, ont été transplantés en 2017, mais il y avait encore 24000 personnes en attente d'un organe la même année. Les organes transplantés peuvent provenir d'un donneur décédé ou, dans le cas des reins et du foie, d'un donneur vivant consentant, le plus souvent membre de la famille du patient. Hélas, même si un proche accepte de prendre ce risque pour sa santé, il ne sera pas forcément compatible avec le patient. Pour cette raison, les pratiques médicales et les législations évoluent dans de nombreux pays afin de permettre la mise en place d'un programme d'échange de dons d'organes.

L'exemple le plus simple d'échange de don d'organes est celui où deux patients $P_1$ et $P_2$ sont accompagnés de donneurs $D_1$ et $D_2$. Les patients sont supposés incompatibles avec les donneurs qui les accompagnent, mais on suppose que $D_1$ est compatible avec $P_2$ et $D_2$ avec $P_1$. Il est alors possible de transplanter un organe de $D_1$ vers $P_2$ et de $D_2$ vers $P_1$ avec le consentement de tous et en suivant la procédure légale.

Plus généralement, un cycle d'échange d'organes associe $k$ paires de patient-donneur $(P_{i_1},D_{i_1}), \dots, (P_{i_k},D_{i_k})$ de sorte que $D_{i_l}$ donne à $P_{i_{l+1}}$ pour $l=1,\dots,k-1$ et $D_{i_k}$ donne à $P_{i_1}$.
Par ailleurs, le point essentiel est que les transferts soient tous réalisés en même temps et dans le même hôpital pour éviter qu'une rétractation de dernière minute ne lèse un patient et son donneur, et que les patients et donneurs venus ensemble et leur famille puissent se soutenir émotionnellement durant l'hospitalisation. 
Pour cette raison, le nombre d'échanges prenant place au sein d'un même cycle est nécessairement limité. En pratique, l'organisation d'un cycle de trois paires est déjà une épreuve pour le personnel d'un hôpital, et le plus grand cycle ayant jamais eu lieu a a impliqué six patients et donneurs.

Dans ce projet, nous prendrons le point de vue de l'organisme national responsable de la gestion du programme d'échange d'organes. 
À chaque phase d'échange, l'objectif de cet organisme est de choisir un ensemble de cycles d'échanges entre paires compatibles afin de maximiser le nombre de patients recevant un organe. Dans certains cas, on peut aussi donner une priorité à certains patients en fonction de la gravité de leur état ou de la durée de leur attente. 
Pour cela, on pourra attribuer des poids différents à chaque patient et maximiser la somme des poids des patients recevant un organe. 

## Étude du problème déterministe


### Jeux de données

Tous vos tests seront basés sur les jeux de données de la [PrefLib](https://www.preflib.org/dataset/00036). Ces jeux de données ne correspondent pas à des programmes d'échanges d'organes réels pour des raisons de confidentialité, mais ils reproduisent la structure des données réelles plus fidèlement que les données aléatoires utilisées pour le projet de RO. Ils 
sont constituées d'informations individuelles et d'un graphe de compatibilité. Chaque fichier .wmd décrit un graphe de compatibilité orienté, $G=(V,A)$, où chaque sommet de $V$ représente une paire patient-donneur et où un arc entre deux paires $(P_k,D_k)$ et $(P_l,D_l)$ signifie que $D_k$ est compatible avec $P_l$. La compatibilité est obtenue à partir des données biologiques individuelles (e.g., les groupes sanguins) et d'un _test croisé_ lors duquel des biologistes mettent en présence des tissus d'un malade et d'un donneur supposé. Chaque fichier est composé comme suit :  
- les 11 premières ligne contiennent diverses informations sur le fichier, dont le nombre de sommets ($n$) et le nombre d'arcs ($m$),
- les $n$ lignes suivantes nomment les sommets (un sommet par paire),
- les $m$ lignes restantes contiennent les arcs du graphe et le poids de chaque arc. Le poids de chaque arc indique l'urgence de la situation du malade de la paire destination. 

Je fournis ci-dessous une fonction permettant de lire les fichiers .wmd et retournant un graphe de compatibilité pondéré par les poids de chaque patient. Pour vous aider à vous lancer, je fournis déjà quelques instances en accompagnement de ce sujet sur Moodle. Elles contiennent un nombre croissant de paires patient-donneur allant de 16 à 512.

In [ ]:
# Include the necessary packages
using Random, Graphs, JuMP, HiGHS, DelimitedFiles, Distributions
include("KEP_readfile.jl")


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [1]:
# Exemple d'utilisation de la fonction supposant que les fichiers sont dans le dossier data_KEP
G, edge_weight = read_wmd_file("data_KEP/KEP_001.wmd");
println("Number of vertices: ", nv(G));
println("Number of arcs: ", ne(G));
println("List of arcs with weights:")
for e in edges(G)
    print("($(e.src),$(e.dst)):$(edge_weight[e.src,e.dst]), ");
end

UndefVarError: UndefVarError: `read_wmd_file` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Formulations compactes

Dans le cadre du TD/TP du cours de Recherche Opérationnelle, il vous a été demandé de coder trois formulations PLNE, à savoir :
- celle sans contrainte sur la longueur des cycles
- celle avec des cycles de taille 2
- celle avec la décomposition par hôpital pour des cycles de taille 2 à 6.

J'ai recopié la correction distribuée sur Moodle dans le fichier formulation_compactes.jl. Je l'inclus ci dessous. Afin de prendre le sujet en main, comparer les solutions des 3 modèles sur un ensemble de jeux de données bien choisis. Analyser les résultats d'un point de vue informatique, pratique et sociétal.

In [7]:
# inclusion des fonctions de résolution des formulations compactes
include("formulations_compactes.jl")

# tests de bon fonctionnement sur le petit graphe importé plus haut
# solve_cycle_infini(G, edge_weight);
# solve_cycle_2(G, edge_weight);
G, edge_weight = read_wmd_file("data_KEP/KEP_071.wmd");
K = 3
solve_cycle_K(G, edge_weight, K);


-------------------------------
Résolution du modèle avec cycle de taille au plus 3

Statut de la résolution : OPTIMAL
Durée de la résolution : 2.259605625
Nombre de transferts réalisés : 47
Liste des transferts réalisés :
2 -> 23 ; 4 -> 17 ; 5 -> 60 ; 6 -> 40 ; 8 -> 41 ; 10 -> 36 ; 11 -> 62 ; 12 -> 57 ; 13 -> 48 ; 14 -> 15 ; 15 -> 27 ; 17 -> 4 ; 18 -> 43 ; 19 -> 33 ; 20 -> 38 ; 22 -> 45 ; 23 -> 2 ; 24 -> 13 ; 25 -> 28 ; 27 -> 14 ; 28 -> 25 ; 30 -> 56 ; 32 -> 35 ; 33 -> 46 ; 35 -> 32 ; 36 -> 63 ; 37 -> 49 ; 38 -> 51 ; 40 -> 61 ; 41 -> 8 ; 42 -> 64 ; 43 -> 52 ; 45 -> 22 ; 46 -> 19 ; 48 -> 24 ; 49 -> 37 ; 51 -> 20 ; 52 -> 18 ; 54 -> 55 ; 55 -> 54 ; 56 -> 30 ; 57 -> 12 ; 60 -> 5 ; 61 -> 6 ; 62 -> 11 ; 63 -> 10 ; 64 -> 42 ; 
-------------------------------



### Génération de colonnes

L'objectif de cette partie du projet est de coder une méthode de génération de colonnes pour le problème de dons d'organes en dominos (KEP pour Kidney Exchange Problem en anglais) sans incertitudes. Votre code s'appuiera sur les éléments vus en CM et leur application au KEP vue en TD. Pour compléter cette présentation, [une page de la documentation de JuMP](https://jump.dev/JuMP.jl/stable/tutorials/algorithms/cutting_stock_column_generation/) est dédiée à la génération de colonnes. Elle vous offre une autre entrée en matière sur la question.

1. Coder une fonction résolvant le modèle par cycles après énumération de touts les cycles de taille $K$ ou moins. Tester pour $K=2,3,4$.
2. Coder une méthode de génération de colonnes pour résoudre la relaxation linéaire de la formulation par cycles. Vous implémenterez deux versions de la fonction de _pricing_ (résolution du sous-problème) : une ou le sous-problème est formulé à l'aide d'un ensemble de PLNE, et une ou le sous-problème est résolu par un algorithme de plus long chemin.
3. Coder une fonction qui résout le problème de dons en dominos de façon approchée à l'aide de l'heuristique de génération de colonnes décrite en CM.


## 1. brute force approach

In [3]:
include("brute_force_approach.jl")
K = 5
G, edge_weight = read_wmd_file("data_KEP/KEP_031.wmd");
C_K = enumerate_cycles(G,K)

brut_force(G,K)

LoadError: LoadError: UndefVarError: `@variable` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at /Users/jeanpousset/Documents/7-Optimisation Stochastique/Projet - Transfert d'organe/brute_force_approach.jl:52
in expression starting at /Users/jeanpousset/Documents/7-Optimisation Stochastique/Projet - Transfert d'organe/brute_force_approach.jl:52

## 2,3. Column generation for Kidney Exchange Problems

In [ ]:
using Random, Graphs, JuMP, HiGHS, DelimitedFiles, Distributions

include("tests.jl")

test01 = KEP_test("data_KEP/KEP_001.wmd"; SP_method="Bellmann")

solve_KEP(test01)



SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
